In [ ]:
!pip install mido

In [ ]:
!pip install pretty_midi

In [ ]:
! pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import os, sys

import mido
import pretty_midi
import re
from konlpy.tag import Okt
okt = Okt()

In [ ]:
lyric_file_dir = "/content/drive/MyDrive/ToBigs/seqGAN논문/lyrics_new/lyrics" ## 경로 수정해주기
midi_file_dir = "/content/drive/MyDrive/ToBigs/seqGAN논문/midi" ## 경로 수정해주기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
lyric_list = os.listdir(lyric_file_dir)

In [ ]:
lyric_list

In [ ]:
len(lyric_list)

In [ ]:
def get_feature0(midi, duration_limit):
    current_time = 0
    feature0_list = []
    
    for msg in midi:

        if msg.type == 'note_on' and msg.velocity > 0 and msg.time > duration_limit:
          start = current_time
          current_time += msg.time
          length = msg.time
          note = msg.note
          freq = 400 * 2**((note-69)/12)
          velocity = msg.velocity
          event = [start, length, freq, velocity]
          feature0_list.append(event)

        elif msg.type == 'note_on' and msg.velocity > 0 and msg.time <= duration_limit:
          current_time += msg.time

        elif msg.type == 'note_off' and msg.velocity > 0:
          current_time += msg.time
                
    return feature0_list

In [ ]:
def x_round(x):
  return round(x*4)/4

In [ ]:
def get_feature1(midi, duration_limit, pretty_data):
    #midi = mido.MidiFile(data_file)

    bpm = pretty_data.estimate_tempo()
    t = 60 / bpm

    feature1_list = []
    duration = 0

    for msg in midi:

        if msg.type == 'note_on' and msg.velocity > 0 and msg.time > duration_limit:
          pitch = msg.note
          note_duration = x_round(msg.time / t)
          rest_duration = duration
          event = [pitch, note_duration, rest_duration]
          feature1_list.append(event)
          duration = 0

        elif msg.type == 'note_on' and msg.velocity > 0 and msg.time <= duration_limit:
          duration += msg.time

        elif msg.type == 'note_off' and msg.velocity > 0:
          duration += msg.time
                
    return feature1_list

In [ ]:
def find_duration_limit(midi):
  for i in [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.12, 0.15, 0.2]:
    feature_list = get_feature0(midi, i)
    k = len(feature_list)
    if(k < 400):
      return i
    if(i == 0.2): # 0.2인데 아직 return 안됐으면 그래도 그냥 0.2로
      return i

In [ ]:
for i in range(330): #midi 개수로 for문 돌리기기
  name = lyric_list[i]
  print(name)
  target_lyrics = lyric_file_dir + "/" + name
  with open(target_lyrics, "r") as f:
    lyrics = f.read()
  goal_lyrics = re.sub(r'[^\s\w]',' ',lyrics)

  name_midi = name.replace('.txt', '.mid')
  target_midi = mido.MidiFile( midi_file_dir + "/" + name_midi)
  pretty_data = pretty_midi.PrettyMIDI(midi_file_dir + "/" + name_midi)

  k = find_duration_limit(target_midi) #적절한 duration 제한을 찾아줌

  feature0_list = get_feature0(target_midi, k)
  feature1_list = get_feature1(target_midi, k, pretty_data)

  len_1 = len(feature1_list)
  print(len_1)

  goal_list = okt.morphs(goal_lyrics)
  feature2_list = []

  for token in goal_list:
    if token != '\n':
      feature2_list.append([token])

  len_2 = len(feature2_list)
  print(len_2)

  if(len_1 < len_2): # if len_1 < len_2 (가사 토큰 개수가 더 큰 경우)
    n = len_2 - len_1
    k = len_2 // n
    l = 0

    if ((n-1)%k == 0):
      for i in range(n-1):
        feature2_list[l].extend(feature2_list[l+1])
        feature2_list[l] = [''.join(feature2_list[l])]
        l += k
    else:
      for i in range(n):
        feature2_list[l].extend(feature2_list[l+1])
        feature2_list[l] = [''.join(feature2_list[l])]
        l += k

    l = 0
    if ((n-1)%k == 0):
      for i in range(n-1):
        del feature2_list[l+1]
        l += k
        l -= 1
    else:
      for i in range(n):
        del feature2_list[l+1]
        l += k
        l -= 1

  if(len_1 > len_2): # if len_1 > len_2 (미디 토큰의 개수가 더 큰 경우)
    n = len_1 - len_2
    k = len_1 // n
    l = 0

    if ((n-1)%k == 0):
      for i in range(n-1):
        del feature0_list[l+1]
        l += k
        l -= 1
    else:
      for i in range(n):
        del feature0_list[l+1]
        l += k
        l -= 1

    l = 0
    if ((n-1)%k == 0):
      for i in range(n-1):
        del feature1_list[l+1]
        l += k
        l -= 1
    else:
      for i in range(n):
        del feature1_list[l+1]
        l += k
        l -= 1

  if (len(feature0_list) == len(feature2_list)+1): #그래도 한개씩 안맞는 경우에는 뒤에서 하나씩만 제거거
    del feature0_list[-1]
  if (len(feature1_list) == len(feature2_list)+1):
    del feature1_list[-1]

  print(len(feature0_list), len(feature1_list), len(feature2_list))

  feature2_list_fin = []

  for i in range(len(feature2_list)):
    feature2_list_fin.extend(feature2_list[i])

  final_array = [[feature0_list, feature1_list, feature2_list_fin]]
  final_array = np.array(final_array)
  
  name_npy = name.replace('.txt', '.npy')

  np.save("/content/drive/MyDrive/ToBigs/1718 NLP_conference/npy_2023/" + name_npy, final_array) #npy 담을 폴더 생성 후

In [ ]:
feature1_list

In [ ]:
feature2_list_fin

### 멈춘부분 인덱스

In [ ]:
n = len(lyric_list)
print(n)

for i in range(n):
  if(lyric_list[i] == "포맨 (4MEN) - 나비무덤.txt"):
    print(i)

In [ ]:
lyric_list[996]

In [ ]:
#1개 제외 -> 노을 보이지않는것들들

In [ ]:
npy_list = os.listdir("/content/drive/MyDrive/ToBigs/1718 NLP_conference/npy_2023")

In [ ]:
len(npy_list)